# This is a description of the method used to generate midi type files, containing pitches of various types

In [1]:
from midiutil import MIDIFile
import random, numpy as np

ModuleNotFoundError: No module named 'midiutil'